In [1]:
################################ErisX BTC-USD Raw To Clean OHLC Data Transformation ##########################
####Importhing the Data#####

import datetime as datetime
import pandas as pd
import requests
import json
import csv


#url = "https://api.pro.coinbase.com/products/BTC-USD/trades?status=COMPLETED&from=2021-11-03&to=2021-12-03"
url = "https://api.pro.coinbase.com/products/BTC-USD/trades"
headers = {'authorization': "Basic API Key Ommitted", 'accept': "application/json", 'accept': "text/csv"}

## API Call to retrieve report
data = requests.get(url, headers=headers).text

## API Results

df = pd.read_json(data)
df.head()


,time,trade_id,price,size,side
0,2021-03-15T09:12:55.88Z,145137186,57498.46,0.001678,sell
1,2021-03-15T09:12:55.065Z,145137185,57498.05,0.047591,buy
2,2021-03-15T09:12:54.404Z,145137184,57500.89,0.008297,sell
3,2021-03-15T09:12:53.772Z,145137183,57506.44,0.085095,buy
4,2021-03-15T09:12:53.505Z,145137182,57531.65,0.137135,buy


In [ ]:
######################################Insight into what I plan to do###############################################

I will first build a fuction that can input any Coin Base raw data and output clean OHLC data.
Following the fucntion is the step by step break down of what I have done to get this function working, that is to clean the RAW data and out put it in the Open-High-Low-Close formate. 

In [2]:
# Single Function To convert Raw data to Clean OHLC data.

def raw_to_clean_data(df):
    #Setting time as index 

    df = df.set_index(pd.DatetimeIndex(df["time"].values))
    df["time"]=pd.to_datetime(df["time"])
    
    #Extracting the Hour information
    df["hour"]=df["time"].dt.hour

    #Finding the volume of trade for each hour
    k=df.groupby(['hour'])['size'].sum()
    k=pd.DataFrame(k)
    k=k.rename({"size":"volume"},axis=1)
    df = df.merge(k,on='hour',how="left")
    df=df.drop(["trade_id","size","side"],axis=1)

    #Finding the Opening price for each hour
    df1=df
    df1 = df1.sort_values('time',ascending = True).groupby('hour').head(1)
    df1=df1.drop(["time","volume"],axis=1)
    df1=df1.rename({"price":"open"},axis=1)
    df = df.merge(df1,on='hour',how="left")

    #Finding the close price for each hour
    df2=df
    df2 = df2.sort_values('time',ascending = False).groupby('hour').head(1)
    df2=df2.drop(["time","volume","open"],axis=1)
    df2=df2.rename({"price":"close"}, axis=1)
    df= df.merge(df2, on="hour", how="left")

    #Finding Highest price for each hour 
    df3=df
    df3 = df3.groupby('hour').agg({'price':['max']})
    #to drop the hysterical nameing price  max 
    df3.columns = df3.columns.droplevel(0)
    df3=df3.rename({"max":"high"},axis=1)
    df=df.merge(df3, on="hour", how="left")
    #Finding Lowest price for each hour 
    df4=df
    df4=df4.groupby("hour").agg({"price":["min"]})
    #to drop the hysterical nameing price  max 
    df4.columns = df4.columns.droplevel(0)
    df4=df4.rename({"min":"low"},axis=1)
    df=df.merge(df4, on="hour", how="left")
    df = df.set_index(pd.DatetimeIndex(df["time"].values))
    df.index=df.index.strftime("%y-%m-%d %H:%00:%00")
    del df["hour"]
    df.to_csv("OHLC_Raw_to_Clean_Data.csv")
    return df


raw_to_clean_data(df)    

,time,price,volume,open,close,high,low
21-03-15 09:0:0,2021-03-15 09:12:55.880000+00:00,57498.46,5.008359,57472.21,57498.46,57546.97,57450.0
21-03-15 09:0:0,2021-03-15 09:12:55.065000+00:00,57498.05,5.008359,57472.21,57498.46,57546.97,57450.0
21-03-15 09:0:0,2021-03-15 09:12:54.404000+00:00,57500.89,5.008359,57472.21,57498.46,57546.97,57450.0
21-03-15 09:0:0,2021-03-15 09:12:53.772000+00:00,57506.44,5.008359,57472.21,57498.46,57546.97,57450.0
21-03-15 09:0:0,2021-03-15 09:12:53.505000+00:00,57531.65,5.008359,57472.21,57498.46,57546.97,57450.0
...,...,...,...,...,...,...,...
21-03-15 09:0:0,2021-03-15 09:12:39.641000+00:00,57472.20,5.008359,57472.21,57498.46,57546.97,57450.0
21-03-15 09:0:0,2021-03-15 09:12:39.641000+00:00,57472.20,5.008359,57472.21,57498.46,57546.97,57450.0
21-03-15 09:0:0,2021-03-15 09:12:39.641000+00:00,57472.20,5.008359,57472.21,57498.46,57546.97,57450.0
21-03-15 09:0:0,2021-03-15 09:12:39.641000+00:00,57472.20,5.008359,57472.21,57498.46,57546.97,57450.0


In [ ]:
##################################The above is the transformed required data#########################################

In [3]:
#######################Step by step explanation of the components of the above function#############################
#Setting time as index 

df = df.set_index(pd.DatetimeIndex(df["time"].values))
df.tail()

,time,trade_id,price,size,side
2021-03-15 09:12:39.641000+00:00,2021-03-15T09:12:39.641Z,145137091,57472.20,0.002233,buy
2021-03-15 09:12:39.641000+00:00,2021-03-15T09:12:39.641Z,145137090,57472.20,0.002211,buy
2021-03-15 09:12:39.641000+00:00,2021-03-15T09:12:39.641Z,145137089,57472.20,0.005754,buy
2021-03-15 09:12:39.641000+00:00,2021-03-15T09:12:39.641Z,145137088,57472.20,0.000004,buy
2021-03-15 09:12:39.641000+00:00,2021-03-15T09:12:39.641Z,145137087,57472.21,0.002165,buy


In [4]:
df["time"]=pd.to_datetime(df["time"])

In [5]:
#Extracting the Hour information
df["hour"]=df["time"].dt.hour

In [6]:
df.head()

,time,trade_id,price,size,side,hour
2021-03-15 09:12:55.880000+00:00,2021-03-15 09:12:55.880000+00:00,145137186,57498.46,0.001678,sell,9
2021-03-15 09:12:55.065000+00:00,2021-03-15 09:12:55.065000+00:00,145137185,57498.05,0.047591,buy,9
2021-03-15 09:12:54.404000+00:00,2021-03-15 09:12:54.404000+00:00,145137184,57500.89,0.008297,sell,9
2021-03-15 09:12:53.772000+00:00,2021-03-15 09:12:53.772000+00:00,145137183,57506.44,0.085095,buy,9
2021-03-15 09:12:53.505000+00:00,2021-03-15 09:12:53.505000+00:00,145137182,57531.65,0.137135,buy,9


In [7]:
#Finding the volume of trade for each hour
k=df.groupby(['hour'])['size'].sum()

In [8]:
k=pd.DataFrame(k)
k

,size
hour,
9,5.008359


In [9]:
k=k.rename({"size":"volume"},axis=1)
df = df.merge(k,on='hour',how="left")

In [10]:
df.head()

,time,trade_id,price,size,side,hour,volume
0,2021-03-15 09:12:55.880000+00:00,145137186,57498.46,0.001678,sell,9,5.008359
1,2021-03-15 09:12:55.065000+00:00,145137185,57498.05,0.047591,buy,9,5.008359
2,2021-03-15 09:12:54.404000+00:00,145137184,57500.89,0.008297,sell,9,5.008359
3,2021-03-15 09:12:53.772000+00:00,145137183,57506.44,0.085095,buy,9,5.008359
4,2021-03-15 09:12:53.505000+00:00,145137182,57531.65,0.137135,buy,9,5.008359


In [11]:
del df["side"]

In [12]:
df=df.drop(["trade_id","size"],axis=1)
df

,time,price,hour,volume
0,2021-03-15 09:12:55.880000+00:00,57498.46,9,5.008359
1,2021-03-15 09:12:55.065000+00:00,57498.05,9,5.008359
2,2021-03-15 09:12:54.404000+00:00,57500.89,9,5.008359
3,2021-03-15 09:12:53.772000+00:00,57506.44,9,5.008359
4,2021-03-15 09:12:53.505000+00:00,57531.65,9,5.008359
...,...,...,...,...
95,2021-03-15 09:12:39.641000+00:00,57472.20,9,5.008359
96,2021-03-15 09:12:39.641000+00:00,57472.20,9,5.008359
97,2021-03-15 09:12:39.641000+00:00,57472.20,9,5.008359
98,2021-03-15 09:12:39.641000+00:00,57472.20,9,5.008359


In [13]:
#Finding the Opening price for each hour
df1=df

In [14]:
df1 = df1.sort_values('time',ascending = True).groupby('hour').head(1)
df1

,time,price,hour,volume
99,2021-03-15 09:12:39.641000+00:00,57472.21,9,5.008359


In [15]:
df1=df1.drop(["time","volume"],axis=1)

In [16]:
df1

,price,hour
99,57472.21,9


In [17]:
df1=df1.rename({"price":"open"},axis=1)

In [18]:
df = df.merge(df1,on='hour',how="left")
df.head()

,time,price,hour,volume,open
0,2021-03-15 09:12:55.880000+00:00,57498.46,9,5.008359,57472.21
1,2021-03-15 09:12:55.065000+00:00,57498.05,9,5.008359,57472.21
2,2021-03-15 09:12:54.404000+00:00,57500.89,9,5.008359,57472.21
3,2021-03-15 09:12:53.772000+00:00,57506.44,9,5.008359,57472.21
4,2021-03-15 09:12:53.505000+00:00,57531.65,9,5.008359,57472.21


In [19]:
df.head()

,time,price,hour,volume,open
0,2021-03-15 09:12:55.880000+00:00,57498.46,9,5.008359,57472.21
1,2021-03-15 09:12:55.065000+00:00,57498.05,9,5.008359,57472.21
2,2021-03-15 09:12:54.404000+00:00,57500.89,9,5.008359,57472.21
3,2021-03-15 09:12:53.772000+00:00,57506.44,9,5.008359,57472.21
4,2021-03-15 09:12:53.505000+00:00,57531.65,9,5.008359,57472.21


In [20]:
#Finding the close price for each hour
df2=df

In [21]:
df2 = df2.sort_values('time',ascending = False).groupby('hour').head(1)
df2.head()

,time,price,hour,volume,open
0,2021-03-15 09:12:55.880000+00:00,57498.46,9,5.008359,57472.21


In [22]:
df2=df2.drop(["time","volume","open"],axis=1)
df2

,price,hour
0,57498.46,9


In [23]:
df2=df2.rename({"price":"close"}, axis=1)
df2.head()

,close,hour
0,57498.46,9


In [24]:
df= df.merge(df2, on="hour", how="left")
df.head()

,time,price,hour,volume,open,close
0,2021-03-15 09:12:55.880000+00:00,57498.46,9,5.008359,57472.21,57498.46
1,2021-03-15 09:12:55.065000+00:00,57498.05,9,5.008359,57472.21,57498.46
2,2021-03-15 09:12:54.404000+00:00,57500.89,9,5.008359,57472.21,57498.46
3,2021-03-15 09:12:53.772000+00:00,57506.44,9,5.008359,57472.21,57498.46
4,2021-03-15 09:12:53.505000+00:00,57531.65,9,5.008359,57472.21,57498.46


In [25]:
#Finsing the Highest and lowest price each hour

In [26]:
#Finding Highest price for each hour 
df3=df
df3 = df3.groupby('hour').agg({'price':['max']})
df3

,price
,max
hour,
9,57546.97


In [27]:
#to drop the hysterical nameing price  max 
df3.columns = df3.columns.droplevel(0)

In [28]:
df3

,max
hour,
9,57546.97


In [29]:
df3=df3.rename({"max":"high"},axis=1)

In [30]:
df3

,high
hour,
9,57546.97


In [31]:
df=df.merge(df3, on="hour", how="left")

In [32]:
df.head()

,time,price,hour,volume,open,close,high
0,2021-03-15 09:12:55.880000+00:00,57498.46,9,5.008359,57472.21,57498.46,57546.97
1,2021-03-15 09:12:55.065000+00:00,57498.05,9,5.008359,57472.21,57498.46,57546.97
2,2021-03-15 09:12:54.404000+00:00,57500.89,9,5.008359,57472.21,57498.46,57546.97
3,2021-03-15 09:12:53.772000+00:00,57506.44,9,5.008359,57472.21,57498.46,57546.97
4,2021-03-15 09:12:53.505000+00:00,57531.65,9,5.008359,57472.21,57498.46,57546.97


In [33]:
#Finding Lowest price for each hour 
df4=df
df4=df4.groupby("hour").agg({"price":["min"]})
#to drop the hysterical nameing price  max 
df4.columns = df4.columns.droplevel(0)
df4=df4.rename({"min":"low"},axis=1)
df=df.merge(df4, on="hour", how="left")
df.head()

,time,price,hour,volume,open,close,high,low
0,2021-03-15 09:12:55.880000+00:00,57498.46,9,5.008359,57472.21,57498.46,57546.97,57450.0
1,2021-03-15 09:12:55.065000+00:00,57498.05,9,5.008359,57472.21,57498.46,57546.97,57450.0
2,2021-03-15 09:12:54.404000+00:00,57500.89,9,5.008359,57472.21,57498.46,57546.97,57450.0
3,2021-03-15 09:12:53.772000+00:00,57506.44,9,5.008359,57472.21,57498.46,57546.97,57450.0
4,2021-03-15 09:12:53.505000+00:00,57531.65,9,5.008359,57472.21,57498.46,57546.97,57450.0


In [34]:
#df.to_csv("OHLC_Clean_Data.csv")